In [4]:
import pandas as pd
import numpy as np
import re
from experiment_utils import get_latest_files
from post_processing import series_name, violin_plots_per_iteration, violin_plots, error_estimation, merge_estimates_true, compute_val_metrics, under_over_analysis, under_over_by_method, val_metrics_per_iteration, boxplots, boxplots_per_iteration, add_weights, exponential_weights_paper

In [2]:
from experiment_utils import get_freq, get_csv_filenames, get_univariate_series
from timecave.data_characteristics import get_features
from statsmodels.tsa.stattools import adfuller, kpss

In [18]:
meteo_files_A = [
    'results/meteorologia/table_A_meteorologia_2024_06_07__15_27_00.csv', #ARMA + Tree
    'results/meteorologia/table_A_meteorologia_LSTM_mltemp_2024_07_31__18_12_10.csv', #LSTM mltemp
    'results/meteorologia/backups/table_A_meteorologia_LSTM_2024_07_31__15_46_31.csv', #LSTM until jena 2
    'results/meteorologia/backups/table_A_meteorologia_LSTM_jena_2024_08_01__05_38_22.csv' #LSTM jena 2 to 6
    ]

meteo_files_B = [
    'results/meteorologia/table_B_meteorologia_2024_06_07__15_27_00.csv', #ARMA + Tree
    'results/meteorologia/table_B_meteorologia_LSTM_mltemp_2024_07_31__18_12_10.csv', #LSTM mltemp
    'results/meteorologia/backups/table_B_meteorologia_LSTM_2024_07_31__15_46_31.csv', #LSTM until jena 2
    'results/meteorologia/backups/table_B_meteorologia_LSTM_jena_2024_08_01__05_38_22.csv' #LSTM jena 2 to 6
]

In [22]:
eng_ciencias_files_A = [
    'results/eng_ciencias/table_A_eng_ciencias_2024_06_06__20_15_36.csv', #ARMA + Tree
    'results/eng_ciencias/table_A_eng_ciencias_LSTM_2024_07_30__15_36_20.csv', #LSTM mechanical gear
    'results/eng_ciencias/backups/table_A_eng_ciencias_LSTM_torque_2024_08_01__03_32_26.csv', #LSTM torque 2 to 5
    'results/eng_ciencias/backups/table_A_eng_ciencias_LSTM_room_torque_2024_07_30__23_44_17.csv', #LSTM room occupancy + torque 0 and 1
    'results/eng_ciencias/table_A_eng_ciencias_LSTM_torque_2_4_2024_07_31__23_39_30.csv', #LSTM torque 6 to 7
    ]

eng_ciencias_files_B = [
    'results/eng_ciencias/table_B_eng_ciencias_2024_06_06__20_15_36.csv', #ARMA + Tree
    'results/eng_ciencias/table_B_eng_ciencias_LSTM_2024_07_30__15_36_20.csv', #LSTM mechanical gear
    'results/eng_ciencias/backups/table_B_eng_ciencias_LSTM_torque_2024_08_01__03_32_26.csv', #LSTM torque 2 to 5
    'results/eng_ciencias/backups/table_B_eng_ciencias_LSTM_room_torque_2024_07_30__23_44_17.csv', #LSTM room occupancy + torque 0 and 1
    'results/eng_ciencias/table_B_eng_ciencias_LSTM_torque_2_4_2024_07_31__23_39_30.csv', #LSTM torque 6 and 7
    ]

In [26]:
energia_files_A = [
    'results/energia/table_A_energia_2024_06_06__17_51_10.csv', #ARMA + Tree
    'results/energia/table_A_energia_LSTM_2024_07_29__19_21_39.csv', #LSTM 
    ]

energia_files_B = [
    'results/energia/table_B_energia_2024_06_06__17_51_10.csv', #ARMA + Tree
    'results/energia/table_B_energia_LSTM_2024_07_29__19_21_39.csv', #LSTM 
    ]

In [61]:
ambiente_files_A = [
    'results/ambiente/table_A_ambiente_2024_06_07__10_15_12.csv', #ARMA + Tree
    'results/ambiente/table_A_ambiente_LSTM_2024_07_29__19_44_35.csv', #LSTM 
    ]

ambiente_files_B = [
    'results/ambiente/table_B_ambiente_2024_06_07__10_15_12.csv', #ARMA + Tree
    'results/ambiente/table_B_ambiente_LSTM_2024_07_29__19_44_35.csv', #LSTM 
    ]

In [31]:
eco_financas_files_A = [
    'results/eco_financas/table_A_eco_financas_2024_06_06__17_36_19.csv', #ARMA + Tree
    'results/eco_financas/table_A_eco_financas_LSTM_2024_07_29__16_38_42.csv', #LSTM 
    ]

eco_financas_files_B = [
    'results/eco_financas/table_B_eco_financas_2024_06_06__17_36_19.csv', #ARMA + Tree
    'results/eco_financas/table_B_eco_financas_LSTM_2024_07_29__16_38_42.csv', #LSTM 
    ]

In [34]:
saude_files_A = [
    'results/saude/table_A_saude_2024_06_06__15_04_07.csv', #ARMA + Tree
    'results/saude/table_A_saude_LSTM_2024_07_29__16_14_31.csv', #LSTM 
    ]

saude_files_B = [
    'results/saude/table_B_saude_2024_06_06__15_04_07.csv', #ARMA + Tree
    'results/saude/table_B_saude_LSTM_2024_07_29__16_14_31.csv', #LSTM 
    ]

In [37]:
transportes_files_A = [
    'results/transportes/table_A_transportes_2024_06_06__10_53_40.csv', #ARMA + Tree
    'results/transportes/table_A_transportes_LSTM_2024_07_30__11_11_37.csv', #LSTM 
    ]

transportes_files_B = [
    'results/transportes/table_B_transportes_2024_06_06__10_53_40.csv', #ARMA + Tree
    'results/transportes/table_B_transportes_LSTM_2024_07_30__11_11_37.csv', #LSTM 
    ]

In [48]:
s1_files_A = [
    'results/s1/table_A_s1_0_200_2024_06_07__11_28_28.csv', #ARMA + Tree
    'results/s1/table_A_s1_200_1001_2024_06_07__12_00_51.csv', #ARMA + Tree
    'results/s1/backups/table_A_s1_0_200_LSTM_2024_06_12__15_47_10.csv', #LSTM 0 to 109,
    'results/s1/backups/table_A_s1_110_200_LSTM_2024_06_12__17_56_42.csv', #LSTM 109 to 199,
    'results/s1/backups/table_A_s1_200_1000_LSTM_2024_06_14__13_47_27.csv', #LSTM 200 to 308,
    'results/s1/backups/table_A_s1_309_1000_LSTM_2024_06_18__16_11_20.csv', #LSTM 309 to 413,
    'results/s1/backups/table_A_s1_414_1000_LSTM_2024_06_19__12_57_51.csv', #LSTM 414 to 520
    'results/s1/backups/table_A_s1_521_1000_LSTM_2024_07_29__16_45_01.csv', #LSTM 521 to 679
    'results/s1/backups/table_A_s1_680_1000_LSTM_2024_07_30__13_07_29.csv', #LSTM 680 to 838
    'results/s1/backups/table_A_s1_839_1000_LSTM_2024_07_30__17_50_55.csv', #LSTM 839 to 996
    'results/s1/table_A_s1_997_1000_LSTM_2024_07_31__13_25_05.csv', #LSTM 996 to 999
    ]

s1_files_B = [
    'results/s1/table_B_s1_0_200_2024_06_07__11_28_28.csv', #ARMA + Tree
    'results/s1/table_B_s1_200_1001_2024_06_07__12_00_51.csv', #ARMA + Tree
    'results/s1/backups/table_B_s1_0_200_LSTM_2024_06_12__15_47_10.csv', #LSTM 0 to 109,
    'results/s1/backups/table_B_s1_110_200_LSTM_2024_06_12__17_56_42.csv', #LSTM 109 to 199,
    'results/s1/backups/table_B_s1_200_1000_LSTM_2024_06_14__13_47_27.csv', #LSTM 200 to 308,
    'results/s1/backups/table_B_s1_309_1000_LSTM_2024_06_18__16_11_20.csv', #LSTM 309 to 413,
    'results/s1/backups/table_B_s1_414_1000_LSTM_2024_06_19__12_57_51.csv', #LSTM 414 to 520
    'results/s1/backups/table_B_s1_521_1000_LSTM_2024_07_29__16_45_01.csv', #LSTM 521 to 679
    'results/s1/backups/table_B_s1_680_1000_LSTM_2024_07_30__13_07_29.csv', #LSTM 680 to 838
    'results/s1/backups/table_B_s1_839_1000_LSTM_2024_07_30__17_50_55.csv', #LSTM 839 to 996
    'results/s1/table_B_s1_997_1000_LSTM_2024_07_31__13_25_05.csv', #LSTM 996 to 999
    ]

In [51]:
s2_files_A = [
    'results/s2/table_A_s2_0_200_2024_06_07__11_38_43.csv', #ARMA + Tree
    'results/s2/table_A_s2_200_1001_2024_06_07__12_11_58.csv', #ARMA + Tree
    'results/s2/backups/table_A_s2_0_200_LSTM_2024_06_12__14_35_25.csv', #LSTM 0 to 80
    'results/s2/backups/table_A_s2_81_200_LSTM_2024_06_12__17_45_25.csv', #LSTM 81 to 175
    'results/s2/backups/table_A_s2_176_200_LSTM_2024_06_14__10_20_50.csv', #LSTM 176 to 199
    'results/s2/backups/table_A_s2_200_1000_LSTM_2024_06_18__15_22_58.csv', #LSTM 200 to 310 
    'results/s2/backups/table_A_s2_311_1000_LSTM_2024_06_20__12_50_57.csv', #LSTM 311 to 421
    'results/s2/backups/table_A_s2_422_1000_LSTM_2024_06_20__16_42_02.csv', #LSTM 422 to 532
    'results/s2/backups/table_A_s2_533_1000_LSTM_2024_06_20__19_08_35.csv', #LSTM 533 to 644 
    'results/s2/backups/table_A_s2_645_1000_LSTM_2024_06_21__12_32_08.csv', #LSTM 645 to 755 
    'results/s2/backups/table_A_s2_756_1000_LSTM_2024_07_29__16_47_25.csv', #LSTM 756 to 913
    'results/s2/table_A_s2_914_1000_LSTM_2024_07_30__11_50_49.csv', #LSTM 914 to 999
    ]

s2_files_B = [
    'results/s2/table_B_s2_0_200_2024_06_07__11_38_43.csv', #ARMA + Tree
    'results/s2/table_B_s2_200_1001_2024_06_07__12_11_58.csv', #ARMA + Tree
    'results/s2/backups/table_B_s2_0_200_LSTM_2024_06_12__14_35_25.csv', #LSTM 0 to 80
    'results/s2/backups/table_B_s2_81_200_LSTM_2024_06_12__17_45_25.csv', #LSTM 81 to 175
    'results/s2/backups/table_B_s2_176_200_LSTM_2024_06_14__10_20_50.csv', #LSTM 176 to 199
    'results/s2/backups/table_B_s2_200_1000_LSTM_2024_06_18__15_22_58.csv', #LSTM 200 to 310 
    'results/s2/backups/table_B_s2_311_1000_LSTM_2024_06_20__12_50_57.csv', #LSTM 311 to 421
    'results/s2/backups/table_B_s2_422_1000_LSTM_2024_06_20__16_42_02.csv', #LSTM 422 to 532
    'results/s2/backups/table_B_s2_533_1000_LSTM_2024_06_20__19_08_35.csv', #LSTM 533 to 644 
    'results/s2/backups/table_B_s2_645_1000_LSTM_2024_06_21__12_32_08.csv', #LSTM 645 to 755 
    'results/s2/backups/table_B_s2_756_1000_LSTM_2024_07_29__16_47_25.csv', #LSTM 756 to 913
    'results/s2/table_B_s2_914_1000_LSTM_2024_07_30__11_50_49.csv', #LSTM 914 to 999
    ]

In [54]:
s3_files_A = [
    'results/s3/table_A_s3_2024_06_07__17_30_48.csv', #ARMA + Tree
    'results/s3/backups/table_A_s3_2024_07_29__16_58_30.csv', #LSTM 0 to 158
    'results/s3/backups/table_A_s3_159_1000_LSTM_2024_07_30__12_49_29.csv', #LSTM 159 to 317
    'results/s3/backups/table_A_s3_318_1000_LSTM_2024_07_30__17_53_55.csv', #LSTM 318 to 476
    'results/s3/backups/table_A_s3_477_1000_LSTM_2024_07_31__16_48_33.csv', #LSTM 477 to 708
    'results/s3/backups/table_A_s3_694_1000_LSTM_2024_07_31__20_30_32.csv', #LSTM 694 to 936
    #'results/s3/table_A_s3_937_1000_LSTM_??????????.csv', #LSTM  to 
    ]

s3_files_B = [
    'results/s3/table_B_s3_2024_06_07__17_30_48.csv', #ARMA + Tree
    'results/s3/backups/table_B_s3_2024_07_29__16_58_30.csv', #LSTM 0 to 158
    'results/s3/backups/table_B_s3_159_1000_LSTM_2024_07_30__12_49_29.csv', #LSTM 159 to 317
    'results/s3/backups/table_B_s3_318_1000_LSTM_2024_07_30__17_53_55.csv', #LSTM 318 to 476
    'results/s3/backups/table_B_s3_477_1000_LSTM_2024_07_31__16_48_33.csv', #LSTM 477 to 708
    'results/s3/backups/table_B_s3_694_1000_LSTM_2024_07_31__20_30_32.csv', #LSTM 694 to 936
    #'results/s3/table_B_s3_937_1000_LSTM_??????????.csv', #LSTM  to 
    ]

In [11]:
def concat_files(files: list[str]):
    dfs = []
    for file in files:
        dfs.append(pd.read_csv(file))
    return pd.concat(dfs, ignore_index=True)


In [62]:
meteo_table_A = concat_files(meteo_files_A)
meteo_table_B = concat_files(meteo_files_B)
eng_ciencias_table_A = concat_files(eng_ciencias_files_A)
eng_ciencias_table_B = concat_files(eng_ciencias_files_B)
energia_table_A = concat_files(energia_files_A)
energia_table_B = concat_files(energia_files_B)
eco_financas_table_A = concat_files(eco_financas_files_A)
eco_financas_table_B = concat_files(eco_financas_files_B)
saude_table_A = concat_files(saude_files_A)
saude_table_B = concat_files(saude_files_B)
transportes_table_A = concat_files(transportes_files_A)
transportes_table_B = concat_files(transportes_files_B)
ambiente_table_A = concat_files(ambiente_files_A)
ambiente_table_B = concat_files(ambiente_files_B)

In [67]:
saude_table_A.isna().sum()

filename        0
column_index    0
method          0
iteration       0
model           0
mse             0
mae             0
rmse            0
dtype: int64

In [55]:
s1_table_A = concat_files(s1_files_A)
s1_table_B = concat_files(s1_files_B)
s2_table_A = concat_files(s2_files_A)
s2_table_B = concat_files(s2_files_B)
s3_table_A = concat_files(s3_files_A)
s3_table_B = concat_files(s3_files_B)

In [63]:
ambiente_table_A

,filename,column_index,method,iteration,model,mse,mae,rmse
0,datasets/processed_data/aire_discharge_environ...,0,<timecave.validation_methods.OOS.Holdout objec...,0,Tree,62.081594,4.285090,7.879187
1,datasets/processed_data/aire_discharge_environ...,0,<timecave.validation_methods.OOS.Holdout objec...,0,ARMA,43.947064,3.932643,6.629258
2,datasets/processed_data/aire_discharge_environ...,0,<timecave.validation_methods.OOS.Repeated_Hold...,0,Tree,60.329963,4.246909,7.767236
3,datasets/processed_data/aire_discharge_environ...,0,<timecave.validation_methods.OOS.Repeated_Hold...,0,ARMA,42.022713,3.898933,6.482493
4,datasets/processed_data/aire_discharge_environ...,0,<timecave.validation_methods.OOS.Repeated_Hold...,1,Tree,60.854273,4.296166,7.800915
...,...,...,...,...,...,...,...,...
3271,datasets/processed_data/forest_fires_Rio_Janei...,0,<timecave.validation_methods.markov.MarkovCV o...,5,LSTM,5506.652331,45.211547,74.206821
3272,datasets/processed_data/forest_fires_Rio_Janei...,0,<timecave.validation_methods.markov.MarkovCV o...,6,LSTM,15954.401677,62.104035,126.310735
3273,datasets/processed_data/forest_fires_Rio_Janei...,0,<timecave.validation_methods.markov.MarkovCV o...,7,LSTM,8004.827173,51.711939,89.469700
3274,datasets/processed_data/forest_fires_Rio_Janei...,0,<timecave.validation_methods.markov.MarkovCV o...,8,LSTM,12534.636870,68.677553,111.958193


In [69]:
#ambiente_table_A.to_csv('results/ambiente/table_A_ambiente.csv', index=False)
ambiente_table_B.to_csv('results/ambiente/table_B_ambiente.csv', index=False)

In [68]:
#meteo_table_A.to_csv('results/meteorologia/table_A_meteorologia.csv', index=False)
#meteo_table_B.to_csv('results/meteorologia/table_B_meteorologia.csv', index=False)

In [24]:
#eng_ciencias_table_A.to_csv('results/eng_ciencias/table_A_eng_ciencias.csv', index=False)
#eng_ciencias_table_B.to_csv('results/eng_ciencias/table_B_eng_ciencias.csv', index=False)

In [30]:
#energia_table_A.to_csv('results/energia/table_A_energia.csv', index=False)
#energia_table_B.to_csv('results/energia/table_B_energia.csv', index=False)

In [33]:
#eco_financas_table_A.to_csv('results/eco_financas/table_A_eco_financas.csv', index=False)
#eco_financas_table_B.to_csv('results/eco_financas/table_B_eco_financas.csv', index=False)

In [36]:
#saude_table_A.to_csv('results/saude/table_A_saude.csv', index=False)
#saude_table_B.to_csv('results/saude/table_B_saude.csv', index=False)

In [39]:
#transportes_table_A.to_csv('results/transportes/table_A_transportes.csv', index=False)
#transportes_table_B.to_csv('results/transportes/table_B_transportes.csv', index=False)

In [46]:
#s1_table_A.to_csv('results/s1/table_A_s1.csv', index=False)
#s1_table_B.to_csv('results/s1/table_B_s1.csv', index=False)

In [53]:
#s2_table_A.to_csv('results/s1/table_A_s2.csv', index=False)
#s2_table_B.to_csv('results/s1/table_B_s2.csv', index=False)